In [53]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pyscf import gto, scf, fci
import numpy as np
import matplotlib.pyplot as plt

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_aer import AerSimulator, AerProvider

from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit import transpile
from qiskit_ibm_runtime import Options

from qiskit import QuantumCircuit, transpile
from qiskit.visualization.timeline import draw
from qiskit.providers.fake_provider import GenericBackendV2
 


In [54]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService.save_account(
    token='OOEf6smekUlKyUFrSF6OSxwLT90zrz1M0lJZWdVgkXO0', # Your token is confidential.
    # Do not share your key in public code.
    instance="crn:v1:bluemix:public:quantum-computing:eu-de:a/624a449c58db4cebbbcd6d44cd96889e:461be4b9-9545-4510-8f57-8a13fe1ec647::",
    name=".", # Optionally name this set of account credentials.
    set_as_default=True, # Optionally set these as your default credentials.
    overwrite=True
  )

In [55]:
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
#backend = AerSimulator(method = "statevector",noise_model=None)

In [56]:
def fermion_to_qubit(problem, second_q_op, mapper_name,  z2symmetry_reduction=None):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()

  qubit_op = mapper.map(second_q_op)

  if z2symmetry_reduction != None:
    mapper = problem.get_tapered_mapper(mapper)
    qubit_op = mapper.map(second_q_op)
  else :
    qubit_op = mapper.map(second_q_op)

  return qubit_op , mapper

def FCI(dist):
    # 1. 분자 정의
    mol = gto.M(
        atom = 'H 0 0 0; H 0 0 {}'.format(dist),  # 수소 원자 2개, 거리 0.74 Å
        basis = 'sto-3g',              # 간단한 기저함수
        unit = 'Angstrom',
        spin = 0,                      # 전자 수 = 2, 총 spin = 0 (singlet)
        charge = 0
    )

    # 2. Hartree-Fock 계산
    mf = scf.RHF(mol)
    hf_energy = mf.kernel()

    # 3. FCI 계산 (Full CI)
    cisolver = fci.FCI(mol, mf.mo_coeff)
    fci_energy, fci_vector = cisolver.kernel()

    return fci_energy

In [57]:
def estimate_runtime(
    circ,
    backend,
    shots=1024,
    rep_delay=None,              # None이면 백엔드 기본값(없으면 250µs로 가정)
    scheduling_method="alap",    # "asap"도 가능
    optimization_level=3,
):
    """
    Returns: dict with
      - circuit_seconds: 회로 1샷 실행 길이(초)
      - rep_delay: 사용된 rep_delay(초)
      - shots: 샷 수
      - rough_total_seconds: 대략 총 실행시간(초) ≈ shots × (회로길이 + rep_delay)
      - transpiled: 스케줄된 트랜스파일 결과(참고용)
    """
    # 0) dt 확보
    dt = getattr(backend, "dt", None)
    if dt is None:
        try:
            dt = backend.configuration().dt
        except Exception:
            dt = None

    # 1) virtual RZ duration=0 보정 (백엔드 InstructionDurations에 추가)
    durations = getattr(backend, "instruction_durations", None)
    try:
        n_qubits = backend.configuration().num_qubits
    except Exception:
        n_qubits = None

    if durations is not None and n_qubits is not None:
        # rz duration이 없어서 나는 오류를 회피: 전 큐빗에 대해 0으로 업데이트
        updates = [("rz", [q], 0) for q in range(n_qubits)]
        try:
            durations.update(updates)
        except Exception:
            pass  # 일부 버전에선 read-only일 수 있음 → 그냥 진행

    # 2) 스케줄 포함 트랜스파일
    tcirc = transpile(
        circ,
        backend=backend,
        scheduling_method=scheduling_method,
        optimization_level=optimization_level,
        instruction_durations=durations,  # 위에서 만든 durations 넘겨줌(가능한 경우)
    )

    # 3) 회로 길이(초) 계산
    #    - 우선 순위: (A) Circuit.duration(스케줄 후 dt-cycles) → (B) estimate_duration(target)
    circuit_seconds = None
    if hasattr(tcirc, "duration") and tcirc.duration is not None and dt:
        # QuantumCircuit.duration 은 dt step 수. 초로 변환
        circuit_seconds = tcirc.duration * dt
    else:
        # Terra 0.25+ 에서는 estimate_duration 사용 가능
        try:
            cycles = tcirc.estimate_duration(backend.target)  # dt step 수
            if dt:
                circuit_seconds = cycles * dt
        except Exception:
            circuit_seconds = None

    # 4) rep_delay 결정
    if rep_delay is None:
        rep_delay = getattr(getattr(backend, "configuration", lambda: None)(), "default_rep_delay", None)
    if rep_delay is None:
        # 못 찾으면 보수적 기본값(250µs)
        rep_delay = 2.5e-4

    # 5) 총 시간(러프) 계산
    rough_total = None
    if circuit_seconds is not None:
        rough_total = shots * (circuit_seconds + rep_delay)

    return {
        "circuit_seconds": circuit_seconds,
        "rep_delay": rep_delay,
        "shots": shots,
        "rough_total_seconds": rough_total,
        "transpiled": tcirc,
    }

In [58]:
atoms = ["H", "H"]
basis = 'sto3g'
dist = 0.735
coords = [(0,0,0), (dist,0,0)]
charge = 0
multiplicity = 1
Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run() # 여기는 이후, As_transformer 로 변경. 
fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
print(second_q_op)
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW", z2symmetry_reduction=None)
print(hamiltonian)
num_particles = E_problem.num_particles
num_spatial_orbitals = E_problem.num_spatial_orbitals 
init_state = HartreeFock(num_spatial_orbitals,num_particles,mapper)
ansatz = UCCSD(num_spatial_orbitals,num_particles,mapper,initial_state=init_state)
#ansatz = TwoLocal(num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=init_state)
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
ansatz_isa = pm.run(ansatz)
hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout) # 그니까 컴파일 된거.

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.2563390730032495 * ( +_0 -_0 )
+ -0.4718960072811421 * ( +_1 -_1 )
+ -1.2563390730032495 * ( +_2 -_2 )
+ -0.4718960072811421 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.3322908651276482 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.3322908651276482 * ( +_0 +_3 -_3 -_0 )
+ 0.0904655998921157 * ( +_0 +_0 -_1 -_1 )
+ 0.0904655998921157 * ( +_0 +_1 -_0 -_1 )
+ 0.0904655998921157 * ( +_0 +_2 -_3 -_1 )
+ 0.0904655998921157 * ( +_0 +_3 -_2 -_1 )
+ 0.0904655998921157 * ( +_1 +_0 -_1 -_0 )
+ 0.0904655998921157 * ( +_1 +_1 -_0 -_0 )
+ 0.0904655998921157 * ( +_1 +_2 -_3 -_0 )
+ 0.0904655998921157 * ( +_1 +_3 -_2 -_0 )
+ 0.3322908651276482 * ( +_1 +_0 -_0 -_1 )
+ 0.34928686136600906 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276482 * ( +_1 +_2 -_2 -_1 )
+ 0.34928686136600906 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.3322908651276482 * ( +_2 +_1 -_1 -_2 )
+ 0.33785507740175

In [61]:
# ── 사용 예시 ──
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

info = estimate_runtime(ansatz_isa, backend, shots=100, rep_delay=2.5e-4)
print(f"회로 길이(1샷): {info['circuit_seconds']:.6e} s")
print(f"대략 총 실행시간(shots 포함): {info['rough_total_seconds']:.6f} s")

회로 길이(1샷): 3.414000e-05 s
대략 총 실행시간(shots 포함): 0.028414 s


/var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/ipykernel_26862/4030347797.py:41: DeprecationWarning: ``qiskit.compiler.transpiler.transpile()``'s argument ``instruction_durations`` is deprecated as of Qiskit 1.3. It will be removed in Qiskit 2.0. The `target` parameter should be used instead. You can build a `Target` instance with defined instruction durations with `Target.from_configuration(..., instruction_durations=...)`
  tcirc = transpile(
/var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/ipykernel_26862/4030347797.py:52: DeprecationWarning: The property ``qiskit.circuit.quantumcircuit.QuantumCircuit.duration`` is deprecated as of qiskit 1.3.0. It will be removed in Qiskit 2.0.0.
  if hasattr(tcirc, "duration") and tcirc.duration is not None and dt:
/var/folders/mq/vvzpq4_16092g__xg10fscp80000gn/T/ipykernel_26862/4030347797.py:54: DeprecationWarning: The property ``qiskit.circuit.quantumcircuit.QuantumCircuit.duration`` is deprecated as of qiskit 1.3.0. It will be removed in Qisk

In [60]:
ansatz_isa.count_ops()

OrderedDict([('rz', 246), ('sx', 161), ('ecr', 49), ('x', 6)])

In [19]:
inst_props = backend.target["sx"]  # (0,1) 큐빗의 CX 게이트 속성

duration_cycles = inst_props.duration  # 사이클 수 (int 또는 float)
if duration_cycles is not None and dt is not None:
    gate_time_sec = duration_cycles * dt
    print(f"CX(0,1) duration: {gate_time_sec*1e9:.1f} ns")
else:
    print("Duration 정보가 없습니다.")

AttributeError: 'dict' object has no attribute 'duration'

In [21]:
print(dt)

5e-10


In [35]:
list(backend.target["x"].keys())

[(0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (6,),
 (7,),
 (8,),
 (9,),
 (10,),
 (11,),
 (12,),
 (13,),
 (14,),
 (15,),
 (16,),
 (17,),
 (18,),
 (19,),
 (20,),
 (21,),
 (22,),
 (23,),
 (24,),
 (25,),
 (26,),
 (27,),
 (28,),
 (29,),
 (30,),
 (31,),
 (32,),
 (33,),
 (34,),
 (35,),
 (36,),
 (37,),
 (38,),
 (39,),
 (40,),
 (41,),
 (42,),
 (43,),
 (44,),
 (45,),
 (46,),
 (47,),
 (48,),
 (49,),
 (50,),
 (51,),
 (52,),
 (53,),
 (54,),
 (55,),
 (56,),
 (57,),
 (58,),
 (59,),
 (60,),
 (61,),
 (62,),
 (63,),
 (64,),
 (65,),
 (66,),
 (67,),
 (68,),
 (69,),
 (70,),
 (71,),
 (72,),
 (73,),
 (74,),
 (75,),
 (76,),
 (77,),
 (78,),
 (79,),
 (80,),
 (81,),
 (82,),
 (83,),
 (84,),
 (85,),
 (86,),
 (87,),
 (88,),
 (89,),
 (90,),
 (91,),
 (92,),
 (93,),
 (94,),
 (95,),
 (96,),
 (97,),
 (98,),
 (99,),
 (100,),
 (101,),
 (102,),
 (103,),
 (104,),
 (105,),
 (106,),
 (107,),
 (108,),
 (109,),
 (110,),
 (111,),
 (112,),
 (113,),
 (114,),
 (115,),
 (116,),
 (117,),
 (118,),
 (119,),
 (120,),
 (121,),
 (122,),
 (1

In [38]:
inst_props = backend.target["ecr"][(1,0)]  # 튜플로 큐빗 지정
duration_cycles = inst_props.duration
if duration_cycles is not None and dt is not None:
    gate_time_sec = duration_cycles 
    #print(duration_cycles)
    print(f"ecr gate duration: {gate_time_sec*1e9:.1f} ns")
else:
    print("Duration 정보가 없습니다.")

ecr gate duration: 660.0 ns


In [40]:
inst_props = backend.target["sx"][(0,)]  # 튜플로 큐빗 지정
duration_cycles = inst_props.duration
if duration_cycles is not None and dt is not None:
    gate_time_sec = duration_cycles 
    print(f"sx gate duration: {gate_time_sec*1e9:.1f} ns")
else:
    print("Duration 정보가 없습니다.")

sx gate duration: 60.0 ns


In [41]:
inst_props = backend.target["x"][(0,)]  # 튜플로 큐빗 지정
duration_cycles = inst_props.duration
if duration_cycles is not None and dt is not None:
    gate_time_sec = duration_cycles 
    print(f"x gate duration: {gate_time_sec*1e9:.1f} ns")
else:
    print("Duration 정보가 없습니다.")

x gate duration: 60.0 ns


In [ ]:
print(hamiltonian_isa)

In [ ]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [ ]:
num_params = ansatz.num_parameters
x0 = 2 * np.pi * np.random.random(num_params) 

In [ ]:
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 2000

    res = minimize(
        cost_func,
        x0,
        args=(ansatz_isa, hamiltonian_isa, estimator),
        method="cobyla",
    )

In [ ]:
E_exact = FCI(dist)


plt.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"] + repulsion, label='E_VQE')
plt.axhline(E_exact, color = 'r', linewidth = 0.8, label = 'Clasical Calc. energy(FCI)')
plt.xlabel("Iterations (#)")
plt.ylabel("Energy (Hartree)")
plt.title("Energy Calc. for each iteration")
plt.legend()
plt.grid()
plt.draw()

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    instance='crn:v1:bluemix:public:quantum-computing:eu-de:a/624a449c58db4cebbbcd6d44cd96889e:461be4b9-9545-4510-8f57-8a13fe1ec647::'
)
job = service.job('d2cqgee7nsps73dnljf0')
result = job.result()

for idx, pub_result in enumerate(result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

In [ ]:
print(
    f"The result of the submitted job had {len(result)} PUB and has a value:\n {result}\n"
)
print(
    f"The associated PubResult of this job has the following data bins:\n {result[0].data}\n"
)
print(f"And this DataBin has attributes: {result[0].data.keys()}")
print(
    "Recall that this shape is due to our array of parameter binding sets having shape (100, 2) -- where 2 is the\n\
         number of parameters in the circuit -- combined with our array of observables having shape (3, 1). \n"
)
print(
    f"The expectation values measured from this PUB are: \n{result[0].data.evs}"
)